In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 # opencv
from mtcnn.mtcnn import MTCNN
from matplotlib import pyplot as plt
from keras.models import load_model
from PIL import Image
import os
from keras import backend
from tensorflow.keras.models import load_model
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import glob
import cv2
import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [2]:
def extract_face(filename, required_size=(160, 160)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    #for i in range(len(results)):
    # extract the bounding box from the first face
    for i in range(len(results)):
        x1, y1, width, height = results[i]['box']
        # deal with negative pixel index
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        # extract the face
        face = pixels[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = np.asarray(image)
        return face_array



In [3]:
def load_face(dir):
    faces = list()
    # enumerate files
    for filename in os.listdir(dir):
        path = dir + filename
        face = extract_face(path)
        faces.append(face)
    return faces

def load_dataset(dir):
    # list for faces and labels
    X, y = list(), list()
    for subdir in os.listdir(dir):
        path = dir + subdir + '/'
        faces = load_face(path)
        labels = [subdir for i in range(len(faces))]
        print("loaded %d sample for class: %s" % (len(faces),subdir) ) # print progress
        X.extend(faces)
        y.extend(labels)
    return np.asarray(X), np.asarray(y)

In [4]:
np.savez_compressed('faces-dataset.npz', trainX, trainy, testX, testy)


NameError: name 'trainX' is not defined

In [5]:
facenet_model = load_model('./facenet_keras.h5')

In [6]:
data = np.load('faces-dataset.npz',allow_pickle=True)

In [7]:
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']

In [8]:
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

Loaded:  (381, 160, 160, 3) (381,) (144, 160, 160, 3) (144,)


In [9]:
def get_embedding(model, face):
    # scale pixel values
    face = face.astype('float32')
    # standardization
    mean, std = face.mean(), face.std()
    face = (face-mean)/std
    # transfer face into one sample (3 dimension to 4 dimension)
    sample = np.expand_dims(face, axis=0)
    # make prediction to get embedding
    yhat = model.predict(sample)
    return yhat[0]

emdTrainX = list()
for face in trainX:
    emd = get_embedding(facenet_model, face)
    emdTrainX.append(emd)    
emdTrainX = np.asarray(emdTrainX)
print(emdTrainX.shape)


emdTestX = list()
for face in testX:
    emd = get_embedding(facenet_model, face)
    emdTestX.append(emd)
emdTestX = np.asarray(emdTestX)
print(emdTestX.shape)
np.savez_compressed('faces-embeddings.npz', emdTrainX, trainy, emdTestX, testy)

(381, 128)
(144, 128)


In [12]:
in_encoder = Normalizer()
emdTrainX_norm = in_encoder.transform(emdTrainX)
emdTestX_norm = in_encoder.transform(emdTestX)

out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy_enc = out_encoder.transform(trainy)
testy_enc = out_encoder.transform(testy)

model = SVC(kernel='linear', probability=True)
pred= model.fit(emdTrainX_norm, trainy_enc)

yhat_train = model.predict(emdTrainX_norm)
yhat_test = model.predict(emdTestX_norm)

score_train = accuracy_score(trainy_enc, yhat_train)
score_test = accuracy_score(testy_enc, yhat_test)

print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=99.213, test=96.528


In [13]:
print(yhat_test)


[ 0  0  0  0  1  1  1  1  1  1  2  2  2  2  2  3  3  3  3  3  3  3  4  4
  4  5  5  5  5  5  6  1  6  6  7  7  7  7  7  7  7  8  8  8  8  8  8  8
  8  9 10 10 10 10 10 11 11 11 12 12 12 12 12 12 12 12 12 12 12 12 13 13
 15 13 14 14 14 14 14 15 15 15 15 15 16 16 16 16 16 17 17 17 17 17 18 18
 18 18 18 19 19 19 19 15 20 20 20 21 21 21 21 21 21 22 22 22 22 23 23 23
 23 23 24 24 24 25 25 25 25 25 26 26 26 27 27 27 27 28 28 28 29 29  8 29]


In [14]:
print(score_test)

0.9652777777777778


In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(testy_enc, yhat_test)

array([[ 4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  7,  

In [35]:
image = Image.open('por.jpg')
image = image.convert('RGB')
pixels = np.asarray(image)
detector = MTCNN()
results = detector.detect_faces(pixels)
for i in range(len(results)):
    x1, y1, width, height = results[i]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize((160,160))
    face_array = np.asarray(image)
    cv2.imwrite('./data/img_{}.jpg'.format(i),face_array)
    
data = list()
for i in glob.glob('data/*.jpg', recursive=True):
    data.append(cv2.imread(i))
emdTestX = list()
for face in data:
    emd = get_embedding(facenet_model, face)
    emdTestX.append(emd)
emdTestX = np.asarray(emdTestX)
emdTestX_norm = in_encoder.transform(emdTestX)
yhat_test = model.predict(emdTestX_norm)
print(yhat_test)
predict_name = out_encoder.inverse_transform(yhat_test)
print(predict_name)

[22]
['Sarawadee']


In [36]:
print(yhat_test[0])

22


In [41]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from matplotlib import pyplot

yhat_prob = model.predict_proba(emdTestX_norm)
class_index = yhat_test[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_test)
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))

Predicted: Sarawadee (49.033)


In [42]:
print(yhat_test[0])

22


In [43]:
import pandas as pd
df = pd.read_csv('score.csv')
df

,no,id,name,score
0,1,6035512001,Alameen,NaN
1,2,6035512002,Apinun,NaN
2,3,6035512003,Arim,NaN
3,4,6035512004,Chanchanok,NaN
4,5,6035512005,Chareef,NaN
5,6,6035512006,Jantapa,NaN
6,7,6035512007,Jetnipat,NaN
7,8,6035512008,Kanda,NaN
8,9,6035512009,Ketsiree,NaN
9,10,6035512010,Nanfa,NaN


In [44]:

for i in range(len(yhat_test)):
    print(yhat_test[i])
    predict_name = out_encoder.inverse_transform(yhat_test)
    #df.at[i,'name'] = predict_name[i]
    df.at[yhat_test[i],'score'] = 1

22


In [45]:
df

,no,id,name,score
0,1,6035512001,Alameen,NaN
1,2,6035512002,Apinun,NaN
2,3,6035512003,Arim,NaN
3,4,6035512004,Chanchanok,NaN
4,5,6035512005,Chareef,NaN
5,6,6035512006,Jantapa,NaN
6,7,6035512007,Jetnipat,NaN
7,8,6035512008,Kanda,NaN
8,9,6035512009,Ketsiree,NaN
9,10,6035512010,Nanfa,NaN


In [46]:
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Sheet1')
writer.save()

In [47]:
html = df.to_html()
print(html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>no</th>
      <th>id</th>
      <th>name</th>
      <th>score</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>6035512001</td>
      <td>Alameen</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>6035512002</td>
      <td>Apinun</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>6035512003</td>
      <td>Arim</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>6035512004</td>
      <td>Chanchanok</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>4</th>
      <td>5</td>
      <td>6035512005</td>
      <td>Chareef</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>5</th>
      <td>6</td>
      <td>6035512006</td>
      <td>Jantapa</td>
      <td>NaN</td>
    </tr>
    <tr>
      <th>6</th>
      <td>7</td>
      <td>6035512007</td>